In [ ]:

!nvidia-smi
!pip install yt-dlp
!git clone https://github.com/IDEA-Research/GroundingDINO.git
!pip install -q -e GroundingDINO/

Mon Jun 12 00:17:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cd GroundingDINO && pip install -q -e .

  Preparing metadata (setup.py) ... done


In [ ]:
!mkdir -p weights && cd weights && wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

In [2]:
import yt_dlp

def download_youtube_video(url, output_path):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': output_path,
        'playliststart': 1,  # Start downloading from the first item
        'postprocessor_args': ['-ss', '0', '-t', '60'],  # Limit to the first 60 seconds
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

video_url = "https://www.youtube.com/watch?v=spxtEt6RaS4"
output_file = "path/to/output2.mp4"
download_youtube_video(video_url, output_file)

[youtube] Extracting URL: https://www.youtube.com/watch?v=spxtEt6RaS4&t=79s
[youtube] spxtEt6RaS4: Downloading webpage
[youtube] spxtEt6RaS4: Downloading android player API JSON
[info] spxtEt6RaS4: Downloading 1 format(s): 137+140
[dashsegments] Total fragments: 1566
[download] Destination: path/to/output(1).f137.mp4
[download] 100% of   15.29GiB in 00:07:07 at 36.61MiB/s                    
[dashsegments] Total fragments: 63
[download] Destination: path/to/output(1).f140.m4a
[download] 100% of  622.18MiB in 00:00:17 at 36.40MiB/s                
[Merger] Merging formats into "path/to/output(1).mp4"
Deleting original file path/to/output(1).f140.m4a (pass -k to keep)
Deleting original file path/to/output(1).f137.mp4 (pass -k to keep)


In [ ]:
%cd /content/GroundingDINO


/content/GroundingDINO


In [ ]:
import cv2
import torch
import numpy as np
import groundingdino.datasets.transforms as T
from PIL import Image, ImageDraw, ImageFont
from groundingdino.util.inference import load_model, load_image, predict, annotate

def load_image_from_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    transform = T.Compose(
        [
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    pil_image = Image.fromarray(frame_rgb).convert("RGB")
    image, _ = transform(pil_image, None)  # 3, h, w
    return pil_image, image

def subsample_frames_and_annotate(video_path, output_path, frame_interval=10):
    model = load_model("groundingdino/config/GroundingDINO_SwinT_OGC.py", "weights/groundingdino_swint_ogc.pth")
    TEXT_PROMPT = " turtle ."
    BOX_THRESHOLD = 0.35
    TEXT_THRESHOLD = 0.25
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    annotated_frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        frame_count += 1
        
        if frame_count % frame_interval == 0:
            image_source, image = load_image_from_frame(frame)
            # print(image_source.shape)
            boxes, logits, phrases = predict(
                model=model,
                image=image,
                caption=TEXT_PROMPT,
                box_threshold=BOX_THRESHOLD,
                text_threshold=TEXT_THRESHOLD
            )
            
            annotated_frame = annotate(image_source=np.array(image_source), boxes=boxes, logits=logits, phrases=phrases)
            annotated_frames.append(annotated_frame)
    
    cap.release()
    
    # Save annotated frames as a video
    print(annotated_frames)
    height, width, _ = annotated_frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, 30, (width, height))
    
    for frame in annotated_frames:
        out.write(frame)
    
    out.release()




    # image_source, image = load_image(IMAGE_PATH)
    # image = torch.from_numpy(frame_rgb).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    # image_source = "video_frame"
    # return image_source, image

# Example usage
video_path = "/content/path/to/output.mp4"
output_path = "/content/path/to/annotated_video.mp4"
subsample_frames_and_annotate(video_path, output_path, frame_interval=10)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:884: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/u